## GradCam & GradCam++ on TL models

In [ ]:
import os
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
from keras import backend as K
import tensorflow as tf

from keras.utils import load_img, img_to_array
from keras.applications.resnet import preprocess_input as resnet50_preprocess_input
from keras.applications.vgg16 import VGG16, preprocess_input as vgg16_preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_v3_preprocess_input

from keras.models import Model, model_from_json

from tf_keras_vis.gradcam_plus_plus import GradcamPlusPlus
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore

In [ ]:
K.set_image_data_format('channels_last') # set the image data format to be (batch_size, height, width, channels), typically used in tensorflow

## Load model

In [ ]:
# Load the model architecture and weights
with open('../models_TL/ResNet50_pretrained.json', 'r') as json_file:
    model_json = json_file.read()
model = model_from_json(model_json)
model.load_weights('../models_TL/ResNet50_pretrained.weights.h5')

model.summary()

In [ ]:
penultimate_layer = 'conv5_block3_out'

In [ ]:
# Preprocess your input image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    img_array = inception_v3_preprocess_input(img_array)
    return img_array

# Load data
def load_data(path, num):
    image_dir = os.path.join(path, "Images")
    label_df = pd.read_excel(os.path.join(path, "Labels.xlsx"))
    
    positive = label_df[label_df['Label'] == 1][:num]
    negative = label_df[label_df['Label'] == 0][:num]
    
    X_pos, X_neg = [], []
    path_pos, path_neg = [], []

    for idx, row in tqdm(positive.iterrows()):
        img_name = row['imgName']
        img_array = preprocess_image(os.path.join(image_dir, img_name))
        X_pos.append(img_array)
        path_pos.append(img_name)
        
    for idx, row in tqdm(negative.iterrows()):
        img_name = row['imgName']
        img_array = preprocess_image(os.path.join(image_dir, img_name))
        X_neg.append(img_array)
        path_neg.append(img_name)
        
    return np.array(X_pos), np.array(X_neg), path_pos, path_neg

In [ ]:
# Input path
input_path = 'path/to/images'
positive, negative, path_pos, path_neg = load_data(input_path, 10)

In [ ]:
# GradCam
def save_heatmap(heatmap, image_path, output_path, alpha=0.4):
    # Load original image (without preprocessing)
    original_img = load_img(image_path, target_size=(224, 224))
    original_img = img_to_array(original_img) / 255.0  # Normalize for plotting
    
    # Resize heatmap to match image size
    heatmap = np.uint8(255 * heatmap)
    
    # first clear the plot
    plt.clf()
    
    # # Display the original image
    plt.imshow(original_img)
    plt.imshow(heatmap, cmap='jet', alpha=alpha)  # Overlay the heatmap with some transparency
    plt.colorbar()  # Show color bar for the heatmap
    plt.axis('off')
    
    # Save the image
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)

def generate_heatmap_image(image_path, output_path, gradcam, score, img_array, penultimate_layer):
    heatmap = gradcam(score, img_array, penultimate_layer=penultimate_layer)
    # Heatmap shape is (1, 224, 224), squeeze it to remove the batch dimension
    heatmap = np.squeeze(heatmap)
    # Plot the result
    save_heatmap(heatmap, image_path, output_path)

def heatmaps(paths, samples, gradcam, score, penultimate_layer, outputs):
    images = 'path/to/images'

    for path, sample in tqdm(zip(paths, samples)):
        image_path = os.path.join(images, path)
        output_path = os.path.join(outputs, path)
        
        generate_heatmap_image(image_path, output_path, gradcam, score, sample, penultimate_layer)

## Gradcam

#### Negative

In [ ]:
outputs = "path/to/outputs"

score = CategoricalScore([0])  # Choose class 1, you can also use CategoricalScore([0]) for class 0

# Create the Gradcam object
gradcam = Gradcam(model, clone=False)

heatmaps(path_neg, negative, gradcam, score, penultimate_layer, outputs)

#### Positive

In [ ]:
outputs = 'path/to/outputs'

score = CategoricalScore([1])  # Choose class 1, you can also use CategoricalScore([0]) for class 0

# Create the Gradcam object
gradcam = Gradcam(model, clone=False)

heatmaps(path_pos, positive, gradcam, score, penultimate_layer, outputs)

## Gradcam++

#### Negative

In [ ]:
outputs = 'path/to/outputs'

score = CategoricalScore([0])  # Choose class 1, you can also use CategoricalScore([0]) for class 0

# Create the Gradcam object
gradcam = GradcamPlusPlus(model, clone=False)

heatmaps(path_neg, negative, gradcam, score, penultimate_layer, outputs)

#### Positive

In [ ]:
outputs = 'path/to/outputs'

score = CategoricalScore([1])  # Choose class 1, you can also use CategoricalScore([0]) for class 0

# Create the Gradcam object
gradcam = GradcamPlusPlus(model, clone=False)

heatmaps(path_pos, positive, gradcam, score, penultimate_layer, outputs)